# 常见场景

## 意图识别

In [1]:
deployment = "gpt-4o"
model = "gpt-4o"

In [2]:

import os
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    # api_version="2023-12-01-preview", # 这个版本无 404 问题
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)


def chat(message: str) -> ChatCompletion:
    return client.chat.completions.create(
        model=deployment,
        messages=message,
        temperature=0.8,
        max_tokens=1000
    )


messages = [
    [
        {"role": "system", "content": "Recognize intent from user's input"},
        {"role": "user", "content": "提醒我明早8点有会议"},
    ],
    [
        {"role": "system", "content": "Recognize intent from user's input"},
        {"role": "user", "content": "订明天早5点北京到上海的飞机"},
    ]
]

for msg in messages:
    res = chat(msg)
    print(res.choices[0].message.content, res.usage.total_tokens)


好的，我会提醒你明早8点有会议。 37
好的，您是需要我帮您预订明天早上5点从北京到上海的航班，对吗？请问您有偏好的航空公司吗？另外，请提供您的姓名和联系方式以便完成预订。 77


# 生成 SQL

In [3]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a software engineer, you can answer the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] """

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)


要计算所有学生英语课程的平均成绩，你需要同时使用 "students" 和 "courses" 表。以下是一个 SQL 查询示例，用于完成这个任务：

```sql
SELECT AVG(s.score) AS avg_english_score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = '英语';
```

这个查询的执行步骤如下：
1. 使用 JOIN 将 "students" 表与 "courses" 表连接起来，条件是 `students.course_id` 等于 `courses.id`。
2. 在 WHERE 子句中，过滤出课程名称为 '英语' 的记录。
3. 使用 `AVG(s.score)` 函数计算筛选出的英语课程记录的平均分数。

查询输出将会是所有学生的英语课程的平均成绩。


# 生成代码友好的提示词

In [4]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

sys_prompt = """Recognize the intent from the user's input and format output as JSON string. 
        The output JSON string includes: "intention", "parameters" """

response = client.chat.completions.create(
    model=deployment,
    temperature=0.8,
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ],
)

print(response.choices[0].message.content, response.usage.total_tokens)

```json
{
  "intention": "set_reminder",
  "parameters": {
    "time": "明早8点",
    "event": "会议"
  }
}
``` 90


In [5]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a software engineer, you can write a SQL string as the answer to the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] """

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)

```sql
SELECT AVG(students.score) AS average_english_score
FROM students
JOIN courses ON students.course_id = courses.id
WHERE courses.name = '英语';
```


# 文本规范异常输出的格式

In [6]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a software engineer, you can write a SQL string as the answer to the user request based on the given tables:
1. table “students“ with the columns [id, name, course_id, score] 
2. table "courses" with the columns [id, name] 
when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request" 
                  """

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": system_prompt},
        # {"role": "user", "content": "计算所有学生英语课程的平均成绩"}
        {"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens=500
)

print(response.choices[0].message.content)

invalid request


# Practice

In [7]:
import os
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)

system_prompt = """You are a helpful assistant. Please analyse user's reviews following this instruction:
    If the reviews are positive, return 'Y'; otherwise return 'N'. """

deployment = "gpt-4o"


def analyze_reviews(reviews: str) -> ChatCompletion:
    resp = client.chat.completions.create(
        model=deployment,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": reviews}
        ],
        max_tokens=500
    )
    return resp

    

In [8]:
reviews = [
    "This product is amazing! I love it.",
    "The service was terrible. I will never buy from them again.",
    "The product is okay. It works, but could be better.",
    "I'm not sure about the product. It seems to work fine.",
    "The customer service was great. They were very helpful.",
    "The product broke after a few days. I'm not sure if it's worth the money.",
    "The product arrived damaged. I'm not sure if it's worth the money.", ]


def show_result(resp: ChatCompletion):
    print(resp.choices[0].message.content, "token: ", resp.usage.total_tokens)
    
for review in reviews:
    show_result(analyze_reviews(review))

Y token:  52
N token:  56
N token:  56
N token:  56
Y token:  54
N token:  60
N token:  57
